In [1]:
import tensorflow.keras as keras
from keras.applications.resnet50 import ResNet50
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from utils import get_dendrite_preprocessed_train_data_tf
import numpy as np

/home/student401/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [24]:
X, Y = get_dendrite_preprocessed_train_data_tf()
# Get training pieces
# x 0
# 0 0
X1, Y1 = X[:, 0:100, 0:100], Y[:]
# 0 x
# 0 0
X2, Y2 = X[:, 100:200, 0:100], Y[:]
# 0 0
# x 0
X3, Y3 = X[:, 0:100, 100:200], Y[:]
# 0 0
# 0 x
X4, Y4 = X[:, 100:200, 100:200], Y[:]

# Stack all the training Xs and Ys
Xtrain, Ytrain = np.vstack([X1, X2, X3, X4]), np.vstack([Y1, Y2, Y3, Y4])

In [25]:
def rearrange_one_to_three(X):
    new_X = np.zeros((X.shape[0], 224, 224, 3))
    new_X[:, :, :, 0] = X[:, :, :, 0]
    new_X[:, :, :, 1] = X[:, :, :, 0]
    new_X[:, :, :, 2] = X[:, :, :, 0]
    return new_X

In [26]:
X = rearrange_one_to_three(X)

In [27]:
X.shape

(20000, 224, 224, 3)

In [6]:
image_size = [224, 224]

In [7]:
res = ResNet50(input_shape=image_size + [3], weights='imagenet', include_top=False)

/home/student401/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [8]:
for layer in res.layers:
    layer.trainable = False

In [9]:
X = Flatten()(res.output)

In [10]:
hidden_out1 = Dense(700, activation='relu')(X)

In [11]:
hidden_out2 = Dense(500, activation='relu')(hidden_out1)

In [12]:
hidden_out3 = Dense(200, activation='relu')(hidden_out2)

In [13]:
prediction = Dense(10, activation='softmax')(hidden_out3)

In [14]:
model = Model(inputs=res.input, outputs=prediction)

In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [32]:
model.compile(
    loss='categorical_crossentropy',
  optimizer='rmsprop',
  metrics=['accuracy']
)

In [ ]:
r = model.fit(x=X, y=Y, validation_split=0.1, epochs=15, batch_size=150)

Train on 18000 samples, validate on 2000 samples
Epoch 1/15
18000/18000 [==============================] - 94s 5ms/step - loss: 12.9481 - acc: 0.1964 - val_loss: 14.6191 - val_acc: 0.0930
Epoch 2/15
 4800/18000 [=======>......................] - ETA: 51s - loss: 12.9113 - acc: 0.1990

In [ ]:
from utils import get_dendrite_preprocessed_test_data_tf

In [ ]:
Xtest, Ytest = get_dendrite_preprocessed_test_data_tf()
# Get training pieces
# x 0
# 0 0
Xtest1, Ytest1 = Xtest[:, 0:100, 0:100], Ytest[:]
# 0 x
# 0 0
Xtest2, Ytest2 = Xtest[:, 100:200, 0:100], Ytest[:]
# 0 0
# x 0
Xtest3, Ytest3 = Xtest[:, 0:100, 100:200], Ytest[:]
# 0 0
# 0 x
Xtest4, Ytest4 = Xtest[:, 100:200, 100:200], Ytest[:]

# Stack all the training Xs and Ys
Xtest, Ytest = np.vstack([Xtest1, Xtest2, Xtest3, Xtest4]), np.vstack([Ytest1, Ytest2, Ytest3, Ytest4])


In [ ]:
Xtest = rearrange_one_to_three(Xtest)

In [ ]:
predictions = model.predict(Xtest)

In [ ]:
np.argmax(predictions[:10], axis=1)

In [ ]:
np.argmax(Ytest, axis=1)